## Gabriela Cohen Hadid

### Using BigQuery for Data Analysis

In this part, we will utilize Google Cloud's BigQuery service for data analysis. Instructions on working with BigQuery were provided in the lesson and video tutorial *BigQuery: Cloud Data Warehouse*. Additional information can be found [here](link).

#### Things to Note:
- BigQuery service provides access to data at a large scale, up to petabytes, using SQL commands.
- Your usage with the free access is limited per month. Be mindful when executing queries, especially those not limited to SELECT queries. Avoid using the `*` command, instead select appropriate columns. You can see the size of data processed for each query after writing it, but before running it.
- In this exercise, we'll use the Stack Overflow database with the name `stackoverflow`. Stack Overflow is a highly useful online community for developers and code users who ask and answer questions about coding challenges they encounter.
- The `stackoverflow` database contains tables:
  - `bigquery-public-data.stackoverflow.posts_questions`
  - `bigquery-public-data.stackoverflow.posts_answers`
  - `bigquery-public-data.stackoverflow.users`
- When referring to tables in BigQuery, use backticks around the table name (this is not necessary in regular SQL).
- For each question, print both the query performed and the received output in JSON format.
- If an additional question is asked (besides the query and its output), provide the answer in free text below the query output.

#### 1.
We want to find the top posts related to questions about the JavaScript language. Write a query that retrieves all posts containing the tag "javascript", regardless of case. Return the `id`, `title`, `tag`, and `score` of each post, sorted by score in descending order.

#### 2.
Write a two-line query:
- The first line returns a table with:
  - Total number of posts for which the tag contains "javascript"
  - Number of posts with at least one answer
  - Average number of answers
  - Number of views
  - Average score of all posts with the tag "javascript"
- The second line does the same for posts related to Java, not JavaScript. In this case, the tag contains the string "java" but not "javascript". The description should be "java". Provide meaningful names for the returned fields.

#### 3.
Perform statistics to determine which days of the week the site is busiest. Show the number of posts uploaded on each day of the week, returning a row for each day in the returned table. Which day has the maximum/minimum number of posts? Is there a difference in the quality of posts between different days?

#### 4.
Now, find all questions related to both JavaScript and Python, along with their answers. Write a query that retrieves all questions containing the word "javascript" in the title and "python" in the title, along with their corresponding answers. Each row in the returned table represents a pair of (question + answer). If a question has multiple answers, it should appear in multiple rows in the returned table. Return the `id`, `title`, `tag`, and `body` of both the question and answer, along with the creation date. Remove all newline characters from the `body` field to prevent issues in further work with the file. Save this table to a file named `stackoverflow_javascript_python_qa.csv`. Display the first few rows sorted by the question's ID.

#### 5.
Suppose a US-based company wants to query all users with the highest reputation in each location across the United States. Write a query that returns the user with the highest reputation for each location (city) in the US, who are experts in JavaScript. Exclude cities where there are no such users. Return the fields: `about_me`, `website_url`, `reputation`, `display_name`, `location`. Sort by reputation in descending order and display the top results.

#### 6.
Finally, examine if there's a relationship between the number of questions/answers per user and their reputation. Group all users into categories based on the number of questions multiplied by 1000 (i.e., between 0-999, 1000-1999, etc.) and calculate the average reputation for each such category. Is there a relationship between the number of questions/answers and reputation?

In [90]:
from sqlite3 import connect
from google.cloud import bigquery
import pandas as pd
import matplotlib as plt

In [91]:
my_project_name = "seraphic-elixir-423318-c5" 
client = bigquery.Client(project=my_project_name)

In [92]:
posts_questions = "bigquery-public-data.stackoverflow.posts_questions"
posts_answers = "bigquery-public-data.stackoverflow.posts_answers"
users = "bigquery-public-data.stackoverflow.users"

## Question 1

In [93]:
QUERY = ("""
    SELECT id, title, tags, score, answer_count
    FROM {}
    WHERE LOWER(tags) LIKE "%javascript%"
    ORDER BY score DESC
    LIMIT 5
""").format(posts_questions)
query_job_1 = client.query(QUERY)
results_1 = query_job_1.result() 
results = list(results_1)  # Transforming the RowIterator into a list to prevent multiple iterations
schema = results_1.schema
df_1 = pd.DataFrame([row.values() for row in results],
                  columns=[field.name for field in schema])
df_1.head()
    

,id,title,tags,score,answer_count
0,5767325,How can I remove a specific item from an array?,javascript|arrays,10953,137
1,178325,How do I check if an element is hidden in jQuery?,javascript|jquery|dom|visibility,8447,65
2,1335851,"What does ""use strict"" do in JavaScript, and w...",javascript|syntax|jslint|use-strict,8264,31
3,503093,How do I redirect to another webpage?,javascript|jquery|redirect,7707,58
4,111102,How do JavaScript closures work?,javascript|function|variables|scope|closures,7623,86


## Question 2

In [94]:
QUERY = ("""
    WITH js_table AS (
        SELECT
            "javascript" as description,
            COUNT(id) as number_of_questions, 
            COUNT(answer_count > 0) as at_least_one_answer,
            AVG(answer_count) as average_number_of_answers,
            AVG(view_count) as average_number_of_views,
            AVG(score) as average_score
        FROM {}
        WHERE LOWER(tags) LIKE "%javascript%"
    ),
    java_table AS (
        SELECT
            "java" as description,
            COUNT(id) as number_of_questions,
            COUNT(answer_count > 0) as at_least_one_answer,
            AVG(answer_count) as average_number_of_answers,
            AVG(view_count) as average_number_of_views,
            AVG(score) as average_score
        FROM {}
        WHERE LOWER(tags) LIKE "%java%" AND LOWER(tags) NOT LIKE "%javascript%"
    )
    SELECT * FROM js_table
    UNION ALL
    SELECT * FROM java_table
""").format(posts_questions, posts_questions)
query_job_2 = client.query(QUERY)
results_2 = query_job_2.result()
results = list(results_2)  # Transforming the RowIterator into a list to prevent multiple iterations
schema = results_2.schema
df_2 = pd.DataFrame([row.values() for row in results],
                    columns=[field.name for field in schema])
df_2


,description,number_of_questions,at_least_one_answer,average_number_of_answers,average_number_of_views,average_score
0,java,1900935,1900935,1.607113,3402.134774,2.247800
1,javascript,2434159,2434159,1.554976,2859.457103,2.079275


## Question 3

The days of the week with the minimum and maximum number of questions posted are Saturday and Wednesday respectively.

Regarding the average quality of the posts, we observe an increase from Monday to Tuesday and a decrease from Thursday to Sunday. It is evident that the quality of posts is, on average, higher on Thursdays, reaching its lowest point on Mondays. We can conclude that the quality is relatively high in the middle of the week.

The popularity of the posts exhibits a similar pattern: increasing from Sunday to Tuesday and decreasing from Thursday to Saturday. The highest popularity is on Thursday, and the lowest is on Sunday. It appears that the average number of views (popularity) and the average score of the questions are positively correlated.

In [95]:
QUERY = ("""
    WITH js_table AS (
        SELECT
            CASE 
                WHEN EXTRACT(DAYOFWEEK FROM creation_date) = 1 THEN 'Sunday'
                WHEN EXTRACT(DAYOFWEEK FROM creation_date) = 2 THEN 'Monday'
                WHEN EXTRACT(DAYOFWEEK FROM creation_date) = 3 THEN 'Tuesday'
                WHEN EXTRACT(DAYOFWEEK FROM creation_date) = 4 THEN 'Wednesday'
                WHEN EXTRACT(DAYOFWEEK FROM creation_date) = 5 THEN 'Thursday'
                WHEN EXTRACT(DAYOFWEEK FROM creation_date) = 6 THEN 'Friday'
                ELSE 'Saturday'
            END AS day_of_week,
            COUNT(id) as number_of_questions, 
            COUNTIF(answer_count > 0) as at_least_one_answer,
            AVG(answer_count) as average_number_of_answers,
            AVG(view_count) as average_number_of_views,
            AVG(score) as average_score
        FROM {}
        WHERE LOWER(tags) LIKE "%javascript%"
        GROUP BY day_of_week
    )
    SELECT * FROM js_table
    ORDER BY CASE 
        WHEN day_of_week = 'Sunday' THEN 1
        WHEN day_of_week = 'Monday' THEN 2
        WHEN day_of_week = 'Tuesday' THEN 3
        WHEN day_of_week = 'Wednesday' THEN 4
        WHEN day_of_week = 'Thursday' THEN 5
        WHEN day_of_week = 'Friday' THEN 6
        ELSE 7
    END
""").format(posts_questions)

query_job_3 = client.query(QUERY)
results_3 = query_job_3.result()
results = list(results_3)  # Transforming the RowIterator into a list to prevent multiple iterations
schema = results_3.schema
df_3 = pd.DataFrame([row.values() for row in results],
                    columns=[field.name for field in schema])
df_3

,day_of_week,number_of_questions,at_least_one_answer,average_number_of_answers,average_number_of_views,average_score
0,Sunday,221613,191690,1.498270,2642.399715,1.964086
1,Monday,376293,322260,1.544297,2855.343198,2.009004
2,Tuesday,410725,352668,1.565347,2924.044003,2.119813
3,Wednesday,418134,359182,1.567658,2890.126495,2.108731
4,Thursday,414798,356302,1.569431,2939.209391,2.152626
5,Friday,371657,320208,1.576394,2874.259750,2.080870
6,Saturday,220939,192004,1.523592,2731.443512,2.043003


## Question 4

There are 4909 rows in the full table.

In [96]:
QUERY = ("""
    SELECT
        questions.id AS question_id,
        questions.title AS question_title,
        questions.tags AS question_tags,
        REPLACE(questions.body, '\\n', '') AS question_body,
        REPLACE(answers.body, '\\n', '') AS answer_body
    FROM {} AS questions
    JOIN {} AS answers
        ON questions.id = answers.parent_id
    WHERE LOWER(questions.title) LIKE "%javascript%" 
        AND LOWER(questions.title) LIKE "%python%"
    ORDER BY questions.id
    LIMIT 5
""").format(posts_questions,posts_answers)

query_job_4 = client.query(QUERY)
results_4 = query_job_4.result()
results = list(results_4)  # Transforming the RowIterator into a list to prevent multiple iterations
schema = results_4.schema
df_4 = pd.DataFrame([row.values() for row in results],
                    columns=[field.name for field in schema])
df_4


,question_id,question_title,question_tags,question_body,answer_body
0,39960,Javascript equivalent of Python's locals()?,javascript|python,<p>In Python one can get a dictionary of all l...,<ul><li><p>locals() - No. </p></li><li><p>glob...
1,39960,Javascript equivalent of Python's locals()?,javascript|python,<p>In Python one can get a dictionary of all l...,"<p>Well, I don't think that there is something..."
2,39960,Javascript equivalent of Python's locals()?,javascript|python,<p>In Python one can get a dictionary of all l...,<p>I seem to remember Brendan Eich commented o...
3,39960,Javascript equivalent of Python's locals()?,javascript|python,<p>In Python one can get a dictionary of all l...,"<p>@e-bartek, I think that window[functionName..."
4,39960,Javascript equivalent of Python's locals()?,javascript|python,<p>In Python one can get a dictionary of all l...,"<p>@pkaeding</p><p>Yes, you're right. <em>wind..."


## Question 5

In [97]:
QUERY = ("""
    WITH formatted_users AS (
        SELECT
            CASE
                WHEN LOWER(location) LIKE '%united states of america%' THEN REPLACE(LOWER(location), 'united states of america', 'USA')
                WHEN LOWER(location) LIKE '%united states%' THEN REPLACE(LOWER(location), 'united states', 'USA')
                WHEN LOWER(location) LIKE '%usa%' THEN REPLACE(LOWER(location), 'usa', 'USA')
                ELSE LOWER(location)
            END AS loc,
            display_name,
            website_url,
            about_me,
            reputation
        FROM {}
        WHERE LOWER(about_me) LIKE "%javascript%" 
            AND (LOWER(location) LIKE "%, usa%" OR LOWER(location) LIKE "%, united states" OR LOWER(location) LIKE "%, united states of america")
    )
    SELECT
        loc as location,
        display_name,
        reputation as best_reputation,
        website_url,
        about_me
    FROM formatted_users
    WHERE reputation = (
        SELECT MAX(reputation)
        FROM formatted_users AS fu
        WHERE fu.loc = formatted_users.loc
    )
    ORDER BY best_reputation DESC
    LIMIT 5
""").format(users)
query_job_5 = client.query(QUERY)
results_5 = query_job_5.result()
results = list(results_5)  # Transforming the RowIterator into a list to prevent multiple iterations
schema = results_5.schema
df_5 = pd.DataFrame([row.values() for row in results],
                    columns=[field.name for field in schema])
df_5


,location,display_name,best_reputation,website_url,about_me
0,"pittsburgh, pa, USA",mgilson,288106,http://None,<p>I used to be a fortran and sometimes C prog...
1,"woodinville, wa, USA",Matt Johnson-Pint,219306,http://codeofmatt.com/,<p>né Matt Johnson. He/him/his (<em>not dude/...
2,"rochester, ny, USA",gen_Eric,217247,https://NTICompass.computer,"<p>Before I introduce myself, I just want to p..."
3,"new jersey, USA",hobbs,211474,http://cleverdomain.org,"<p>I'm a programmer, occasional sysadmin, Unix..."
4,"ny, USA",Dave Newton,157222,https://blog.makersend.com,"<p>Ruby/JavaScript &amp; generalist; web, apps..."


## Question 6

Looking at the output data, it's apparent that the number of questions is positively correlated with the average reputation. In other words, as the number of questions increases, so does the average reputation. However, the same phenomenon isn't observed regarding the number of answers. In the output for answers, we see that from the first interval (0-999) to the 12,000-12,999 interval, the average reputation increases, indicating a positive correlation. However, from that interval onward, the average reputation fluctuates in both directions, sometimes increasing and sometimes decreasing with the number of answers per interval.

In [98]:
QUERY = ("""
WITH rep AS (
  SELECT
    users.id,
    users.reputation,
    FLOOR(COUNT(questions.id)/1000) AS categorical_questions
  FROM {} AS users,
    {} AS questions
  WHERE 
    users.id = questions.owner_user_id
  GROUP BY users.id, users.reputation
)
SELECT 
  CAST((categorical_questions * 1000) AS STRING) || '-' || CAST((categorical_questions * 1000 + 999) AS STRING) AS interval_questions,
  AVG(reputation) AS avg_reputation
FROM 
  rep
GROUP BY 
  categorical_questions
ORDER BY categorical_questions;
""").format(users, posts_questions)

query_job = client.query(QUERY)
results = query_job.result()
results_list = list(results)  # Transforming the RowIterator into a list to prevent multiple iterations
schema = results.schema
df = pd.DataFrame([row.values() for row in results_list],
                  columns=[field.name for field in schema])
print(df)


  interval_questions  avg_reputation
0              0-999      349.286313
1          1000-1999    60588.092105
2          2000-2999   109321.500000


In [99]:
QUERY = ("""
WITH reputation_ans AS (
  SELECT
    users.id,
    users.reputation,
    FLOOR(COUNT(answers.id)/1000) AS categorical_answers
  FROM {} AS users,
    {} AS answers
  WHERE 
    users.id = answers.owner_user_id
  GROUP BY users.id, users.reputation
)
SELECT 
  CAST((categorical_answers * 1000) AS STRING) || '-' || CAST((categorical_answers * 1000 + 999) AS STRING) AS interval_answers,
  AVG(reputation) AS avg_reputation
FROM 
  reputation_ans
GROUP BY 
  categorical_answers
ORDER BY categorical_answers;
""").format(users, posts_answers)

query_job = client.query(QUERY)
results = query_job.result()
results_list = list(results)  # Transforming the RowIterator into a list to prevent multiple iterations
schema = results.schema
df = pd.DataFrame([row.values() for row in results_list],
                  columns=[field.name for field in schema])
print(df)


   interval_answers  avg_reputation
0             0-999    4.980740e+02
1         1000-1999    6.422727e+04
2         2000-2999    1.133490e+05
3         3000-3999    1.702245e+05
4         4000-4999    1.993365e+05
5         5000-5999    2.325873e+05
6         6000-6999    2.847613e+05
7         7000-7999    3.155747e+05
8         8000-8999    3.181476e+05
9         9000-9999    3.810306e+05
10      10000-10999    3.813049e+05
11      11000-11999    4.157321e+05
12      12000-12999    5.385009e+05
13      13000-13999    4.581015e+05
14      14000-14999    3.593710e+05
15      15000-15999    6.535228e+05
16      16000-16999    7.518703e+05
17      17000-17999    1.053483e+06
18      19000-19999    7.812150e+05
19      20000-20999    4.690130e+05
20      21000-21999    1.003275e+06
21      22000-22999    9.726395e+05
22      23000-23999    8.748090e+05
23      25000-25999    7.307500e+05
24      28000-28999    9.563005e+05
25      30000-30999    6.944140e+05
26      35000-35999    1.357